### Importing Set Up - using Python 3.11.5

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from IPython.display import display

### Database Creation

In [ ]:
execfile("Setup_and_Organization/Database Setup.py")

### SQLite Database Connection

In [ ]:
engine = create_engine("sqlite:///Resources/database.sqlite")
conn=engine.connect()

# reflect an existing database into a new model
base = automap_base()
# reflect the tables
# base.prepare(engine,reflect=True)

base.prepare(autoload_with=engine)

### Data File Loading

In [14]:
data = pd.read_sql(
f"SELECT * FROM person AS p \
    INNER JOIN occupation AS o ON o.id = p.occupation_id\
    INNER JOIN gender AS g ON p.gender_id = g.id\
    INNER JOIN health AS h ON h.person_id = p.id\
    INNER JOIN bmi AS b ON b.id = h.bmi_id",
    conn
    )

data

,index,id,gender_id,age,occupation_id,index,id,occupation_name,index,id,...,person_id,blood_pressure,stress_level,heart_rate,systolic,diastolic,bmi_id,index,id,bmi_name
0,0,1,0,27,0,0,0,Software Engineer,0,0,...,1,126/83,6,77,126,83,0,0,0,Overweight
1,1,6,0,28,0,0,0,Software Engineer,0,0,...,6,140/90,8,85,140,90,2,2,2,Obese
2,2,85,0,35,0,0,0,Software Engineer,0,0,...,85,120/80,5,70,120,80,1,1,1,Normal
3,3,93,0,35,0,0,0,Software Engineer,0,0,...,93,120/80,5,70,120,80,1,1,1,Normal
4,4,2,0,28,1,1,1,Doctor,0,0,...,2,125/80,8,75,125,80,1,1,1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,369,69,1,33,7,7,7,Scientist,1,1,...,69,128/85,6,76,128,85,0,0,0,Overweight
370,370,70,1,33,7,7,7,Scientist,1,1,...,70,128/85,6,76,128,85,0,0,0,Overweight
371,371,81,1,34,7,7,7,Scientist,1,1,...,81,131/86,8,81,131,86,0,0,0,Overweight
372,372,82,1,34,7,7,7,Scientist,1,1,...,82,131/86,8,81,131,86,0,0,0,Overweight


### Bryan's Visualization / Interaction

In [ ]:
data = pd.read_sql(
f"SELECT * FROM person AS p \
    INNER JOIN occupation AS o ON o.id = p.occupation_id\
    INNER JOIN gender AS g ON p.gender_id = g.id\
    INNER JOIN health AS h ON h.person_id = p.id\
    INNER JOIN bmi AS b ON b.id = h.bmi_id",
    conn
    )

# Possible numerical columns to choose instead of just 'age'

dd = widgets.Dropdown(
    options = {"Age": "age", "Stress Level": "stress_level", "Heart Rate": "heart_rate", "Systolic": "systolic", "Diastolic": "diastolic"},
    value = "age",
    description = "Category"
)

# genders = data["gender_name"].unique().tolist()

dd1 = widgets.Dropdown(
    options = ["All", "Male", "Female"],
    value = "All",
    description = "Gender"
)

# bmis = data["bmi_name"].unique().tolist()

dd2 = widgets.Dropdown(
    options = ["All", "Normal", "Overweight", "Obese"],
    value = "All",
    description = "BMI Type"
)

def draw_plot(column, item1, item2):    
    
    if (item1 == "All"):
        if (item2 == "All"):
            mask = (data["gender_name"].isin(["Male", "Female"])) & (data["bmi_name"].isin(["Normal", "Overweight", "Obese"]))
        else:
            mask = (data["gender_name"].isin(["Male", "Female"])) & (data["bmi_name"] == item2)
    else:
        if (item2 == "All"):
            mask = (data["gender_name"] == item1) & (data["bmi_name"].isin(["Normal", "Overweight", "Obese"]))
        else:
            mask = (data["gender_name"] == item1) & (data["bmi_name"] == item2)
    #sns.set_theme(style = "ticks")
    f, ax = plt.subplots(figsize = (8, 6))
    sns.boxplot(x = data.loc[mask, column],
                y = data.loc[mask, "occupation_name"], 
                hue = data["occupation_name"],
                palette = "Paired",
                width = 0.6,
                legend = False
    )
    sns.stripplot(
        x = data.loc[mask, column],
        y = data.loc[mask, "occupation_name"],
        color = "black",
        dodge = True,
        edgecolor = "black"
    )

    sns.set_style(rc = {"axes.facecolor": "lightyellow"})

    column_proper = column.replace("_", " ").title()
    ax.set_xlabel(column_proper, fontsize = 15)
    ax.set_ylabel("Occupation", fontsize = 15)
    plt.title(f"Distribution of {column_proper} per Occupation ({item1})", fontsize = 18)

ddbox = widgets.HBox([dd, dd1, dd2])
out1 = widgets.interactive_output(draw_plot, {"column": dd, "item1": dd1, "item2": dd2})
display(ddbox, out1)

### Reed's Visualization / Interaction

In [ ]:
#Build the database with the info i would like to use
Person= pd.read_sql("SELECT * FROM person", conn)
Sleep= pd.read_sql("SELECT * FROM sleep", conn)
Health= pd.read_sql("SELECT * FROM health", conn)
merged_data = pd.merge(Person, Sleep, left_on='id', right_on = "person_id")
merged_data = pd.merge(merged_data, Health, left_on='person_id', right_on = "person_id")
data = merged_data[['gender_id', 'age', 'occupation_id', 'sleep_duration', 'sleep_quality', 'stress_level']]
data

In [ ]:
# Create a dropdown widget to select gender
gender_dropdown = widgets.Dropdown(
    options={'Male': 0, 'Female': 1},
    description='Gender:'
)
def draw_plot2(item):
    sns.relplot(x=data.loc[data["gender_id"] == item, "age"],
                y=data.loc[data["gender_id"]== item,"sleep_duration"],
                hue=data.loc[data["gender_id"]== item,"stress_level"],
                size=data.loc[data["gender_id"]== item,"sleep_quality"],
                sizes=(40, 400),
                alpha=.5,
                palette="Paired",
                height=6,
                data=data)\
            .set(title= "Sleep Durration vs Age")
out1= widgets.interactive_output(draw_plot2,{"item":gender_dropdown})
display(gender_dropdown,out1)

### Andrew's Visualization / Interaction

In [34]:
df = pd.read_sql('SELECT * FROM person AS p\
                INNER JOIN sleep AS s\
                ON s.person_id = p.id\
                INNER JOIN occupation AS o\
                ON o.id = p.occupation_id\
                INNER JOIN gender AS g\
                ON p.gender_id = g.id\
                INNER JOIN sleep_disorder AS sd\
                ON sd.id = s.sleep_disorder_id\
                INNER JOIN health AS h\
                ON h.person_id = p.id',
                conn)  # Change 'sleep' to the appropriate table name
# Set a custom color palette
sns.set_palette("husl")
# 1. Sleep Duration/Quality vs Occupation
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=df, x='occupation_name', y='sleep_duration', hue='sleep_quality')
plt.title('Sleep Duration/Quality vs Occupation')
plt.xlabel('Occupation')
plt.ylabel('Sleep Duration')
# Rotate the job titles by 45 degrees
plt.xticks(rotation=45)
# Customize legend labels as numbers from 0 to 10 and legend title
legend_labels = [f'{label}/10' for label in range(11)]
legend_title = 'Quality of Sleep out of 10'
legend = plt.legend(title=legend_title, loc='upper left', fontsize='small', bbox_to_anchor=(1, 1))
if len(legend_labels) == len(legend.get_texts()):
    for i, text in enumerate(legend_labels):
        legend.get_texts()[i].set_text(text)
plt.tight_layout()
# 2. Gender/Age vs Sleep Duration/Quality
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=df, x='gender_name', y='sleep_duration', hue='age')
plt.title('Gender/Age vs Sleep Duration')
plt.xlabel('Gender')
plt.ylabel('Sleep Duration')
legend = plt.legend(title='Age', loc='upper left', fontsize='small', bbox_to_anchor=(1, 1))
legend.set_title('Age', prop={'size': 'small'})
plt.tight_layout()
# 3. Gender/Age vs Quality of Sleep
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=df, x='gender_name', y='sleep_quality', hue='age')
plt.title('Gender/Age vs Quality of Sleep')
plt.xlabel('Gender')
plt.ylabel('Quality of Sleep')
legend = plt.legend(title='Age', loc='upper left', fontsize='small', bbox_to_anchor=(1, 1))
legend.set_title('Age', prop={'size': 'small'})
plt.tight_layout()
# 4. Sleep Duration/Quality vs Sleep Disorder
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='sleep_disorder_name', y='sleep_duration')
plt.title('Sleep Duration vs Sleep Disorder')
plt.xlabel('Sleep Disorder')
plt.ylabel('Sleep Duration')
plt.xticks(rotation=45)
plt.tight_layout()
# 5. Quality of Sleep vs Sleep Disorder
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='sleep_disorder_name', y='sleep_quality')
plt.title('Quality of Sleep vs Sleep Disorder')
plt.xlabel('Sleep Disorder')
plt.ylabel('Quality of Sleep')
plt.xticks(rotation=45)
plt.tight_layout()
# 6. Stress Level vs Occupation
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='occupation_name', y='stress_level')
plt.title('Stress Level vs Occupation')
plt.xlabel('Occupation')
plt.ylabel('Stress Level')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

IndentationError: expected an indented block after 'if' statement on line 27 (2202270793.py, line 28)

### Tom's Visualization / Interaction